# Exercise 1

Autors: Marten Ringwelski, Nico Ostermann, Simon Liessem

Note that this notebook MUST be executed in order to get everything to work.
The tasks can't be run individually. 

Also eCampus does not allow for uploading nested directory structures which makes it hard to properly organize the files. The files are expected to be in the `data` directory which itself is placed next to this notebook.

If you extract the zip file we handed in everything should work just fine.

## a)

Import all we weed and more.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

Set seaborn default theme

In [ ]:
sns.set_theme()

If needed tweak parameters of matplotlib.
Here we increase the size and dpi to bet a bigger but still high-res image.

In [ ]:
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams["figure.figsize"] = (20,15)
%matplotlib inline

In [ ]:
df = pd.read_csv("data/solar_panel.csv")
# I don't like uppercase
df.rename(
    mapper=str.lower,
    axis=1,
    inplace=True,
)

Using proper datetime makes the visualisation nicer.

In [ ]:
df["datetime"] = pd.to_datetime(
    df[["year", "month", "day"]],
)

In [ ]:
_ = df.drop(columns="datetime").hist()

## b)

In [ ]:
groupby_df = df.groupby(
    by="datetime",
)

In [ ]:
power_generated_ser = groupby_df["power generated"].sum()
_ = power_generated_ser.plot(
    kind="line",
    ylabel=power_generated_ser.name,
)

We see in the graph that the maximum power generated is somewhen in May 2009.
This is confirmed by the data.

In [ ]:
power_generated_ser.idxmax()

## c)

First aggregate our data.

In [ ]:
summed_df = groupby_df[["power generated", "sky cover"]].aggregate(
    func={
        "power generated": np.sum,
        "sky cover": np.mean,
    },
)

We split the data (exactly) in half by using the median.

In [ ]:
min_sky_cover = summed_df["sky cover"].min()
max_sky_cover = summed_df["sky cover"].max()
median_sky_cover = summed_df["sky cover"].median()

In [ ]:
summed_df["cluster"] = pd.cut(
    summed_df["sky cover"],
    bins=[min_sky_cover, median_sky_cover, max_sky_cover],
    labels=["low", "high"],
    include_lowest=True,
    right=True,
)

Now show a scatterplot with the clusters displayed in different color.

In [ ]:
fig = plt.figure()
ax = fig.subplots()
for cluster in ["low", "high"]:
    cluster_df = summed_df[summed_df["cluster"] == cluster]

    path = ax.scatter(
        x=cluster_df.index,
        y=cluster_df["power generated"],
    )
    path.set_label(f"{cluster} sky cover")

ax.set_ylabel("Power Generated")
_ = ax.legend()

Now do another scatterplot.

In [ ]:
fig = plt.figure()
ax = fig.subplots()
ax.set_ylabel("Power Generated")
coll = ax.scatter(
    data=summed_df,
    x="cluster",
    y="power generated",
)

# Add some jitter
jitter = 0.1

offsets = coll.get_offsets()
offsets[:, 0] += np.random.uniform(-jitter, jitter, offsets.shape[0])
coll.set_offsets(offsets)
xmin, xmax = ax.get_xlim()
_ = ax.set_xlim(xmin-jitter, xmax+jitter)


We can also show a split violin plot to see the differences in distribution.

In [ ]:
fig = plt.figure()
ax = fig.subplots()

# Work around seaborn limitation
plot_df = summed_df
plot_df["seaborn_hack"] = True
ax.get_xaxis().set_visible(False)

_ = sns.violinplot(
    data=plot_df,
    x="seaborn_hack",
    y="power generated",
    split=True,
    hue="cluster",
    ax=ax,
)